In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.common.by import By

# Khởi tạo trình duyệt (ở đây sử dụng Chrome)
driver = webdriver.Chrome('chromedriver.exe')

# Mở trang web
driver.get("https://hashnode.com/n/hadoop")
time.sleep(2)
# Lấy vị trí cuộn trang ban đầu
last_position = driver.execute_script("return window.pageYOffset;")

while True:
    # Cuộn trang xuống dưới
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
    
    # Đợi 1 giây để trang cuộn xuống
    time.sleep(1)
    
    # Lấy vị trí cuộn trang hiện tại
    current_position = driver.execute_script("return window.pageYOffset;")
    
    # Kiểm tra nếu vị trí cuộn trang không thay đổi, tức đã cuộn đến cuối trang
    if last_position == current_position:
        break
    
    # Cập nhật vị trí cuộn trang cuối cùng
    last_position = current_position

# Đóng trình duyệt
list_link = []
links=driver.find_elements(By.XPATH, '//*[@id="__next"]/div[1]/main/div[2]/div[2]/div/div[2]/div[1]/h1/a')
list_link = [i.get_attribute("href") for i in links]
print(len(list_link))
driver.quit()


WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [63]:
headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'
        }

In [64]:
import requests
from bs4 import BeautifulSoup
import json
from tqdm.notebook import tqdm

session = requests.Session()
session.max_redirects = 50

sum_all = 0
tag = 0
data = []

for i in tqdm(list_link, desc='Crawling ...'):
    try:
        redirect_count = 0
        while redirect_count < 50:
            st = session.get(i, headers=headers, timeout=10, allow_redirects=False)
            if st.status_code == 200:
                break
            elif st.status_code == 302:
                redirect_count += 1
                i = st.headers['Location']
            else:
                print(f"Failed to access {i} with status code {st.status_code}")
                break
        
        if st.status_code == 200:
            result = BeautifulSoup(st.content, 'html.parser')
            content = result.find('div', attrs={'id':'post-content-wrapper'})
            if content is None:
                continue
            title =  result.find('h1', attrs={'data-query':'post-title'})
            if title is None:
                continue
            datas = {
                "url": i,
                "title": title.text.strip(),
                "content": content.text.strip(),
                "code": f"hashnode_hadoop_{tag}",
                "language": 'English',
                "tags": "hadoop"
            }
            data.append(datas)
            tag += 1
            sum_all += 1
    except requests.exceptions.Timeout:
        print(f"Timeout occurred while accessing {i}")
        continue

with open('data6.json', 'r', encoding='utf-8-sig') as f:
    existing_data = json.load(f)

existing_data.extend(data)

with open('data6.json', 'w', encoding='utf-8') as f:
    json.dump(existing_data, f, indent=4, ensure_ascii=False)


Crawling ...:   0%|          | 0/75 [00:00<?, ?it/s]

In [65]:
sum_all

75